# Inicializando funções e variáveis do processo de ETL

In [42]:
%run ./SETUP

Exception: File `'./SETUP.py'` not found.

# Extraindo arquivos

In [32]:
extract_zip(ano_mes)

-1 / unknown

In [33]:
extract_csv(ano_mes)

In [44]:
arquivos_cnes = ['tbCargaHorariaSus', 'rlEstabServClass', 'tbAtividadeProfissional', 'tbClassificacaoServico',\
                 'tbDadosProfissionalSus', 'tbEstabelecimento', 'tbMunicipio']

for file in arquivos_cnes:
    file = get_csv(file, ano_mes)

# Transformando arquivos finais

In [45]:
cond_sp = [tbEstabelecimento.CO_MUNICIPIO_GESTOR == tbMunicipio.CO_MUNICIPIO,
           tbEstabelecimento.CO_ESTADO_GESTOR == 35] #SP

cond_serv = [rlEstabServClass.CO_SERVICO == tbClassificacaoServico.CO_SERVICO_ESPECIALIZADO,
        rlEstabServClass.CO_CLASSIFICACAO == tbClassificacaoServico.CO_CLASSIFICACAO_SERVICO]

estab_munic = tbEstabelecimento.join(tbMunicipio, cond_sp)


df_serv = rlEstabServClass\
            .join(tbClassificacaoServico, cond_serv)\
            .join(estab_munic, rlEstabServClass.CO_UNIDADE == estab_munic.CO_UNIDADE)\
            .select(rlEstabServClass.CO_UNIDADE,
                rlEstabServClass.CO_SERVICO,
                rlEstabServClass.CO_CLASSIFICACAO,
                tbClassificacaoServico.DS_CLASSIFICACAO_SERVICO)\
            .withColumn('DATA_INGESTAO', to_date(current_timestamp()))

In [46]:
df_final = tbCargaHorariaSus\
    .join(tbAtividadeProfissional, tbCargaHorariaSus.CO_CBO ==tbAtividadeProfissional.CO_CBO)\
    .join(estab_munic, tbCargaHorariaSus.CO_UNIDADE == estab_munic.CO_UNIDADE)\
    .join(tbDadosProfissionalSus, tbCargaHorariaSus.CO_PROFISSIONAL_SUS == tbDadosProfissionalSus.CO_PROFISSIONAL_SUS)\
    .select(tbCargaHorariaSus.CO_UNIDADE,
            tbCargaHorariaSus.CO_PROFISSIONAL_SUS,
            tbDadosProfissionalSus.NO_PROFISSIONAL,
            tbCargaHorariaSus.CO_CBO,
            tbCargaHorariaSus.TP_SUS_NAO_SUS,            
            tbAtividadeProfissional.DS_ATIVIDADE_PROFISSIONAL,
            estab_munic.NO_FANTASIA,
            estab_munic.NO_BAIRRO,
            estab_munic.NO_MUNICIPIO,
            estab_munic.CO_MUNICIPIO,
            estab_munic.CO_SIGLA_ESTADO,
            estab_munic.CO_CEP,
            )\
    .withColumn('DATA_INGESTAO', to_date(current_timestamp()))

# Carregando arquivos finais

In [37]:
write_curated_file(df_final, 'curated_estabelecimentos', ano_mes)

In [38]:
write_curated_file(df_serv, 'curated_servicos', ano_mes)

In [39]:
update_table('curated_servicos', 'curated_servicos', ano_mes)

Table curated_servicos updated
PostgreSQL connection is closed


In [40]:
update_table('curated_estabelecimentos', 'curated_estabelecimentos', ano_mes)

Table curated_estabelecimentos updated
PostgreSQL connection is closed
